In [1]:
import requests
from bs4 import BeautifulSoup


In [63]:
r= requests.get("https://pythonizing.github.io/data/real-estate/rock-springs-wy/LCWYROCKSPRINGS/")

c=r.content

soup=BeautifulSoup(c, 'html.parser')

all=soup.find_all("div", {"class":"propertyRow"})

#for getting page number from website
page_number= soup.find_all("a", {"class":"Page"})
last_page=page_number[-1].text
print(last_page)



for item in all:  
    price=item.find("h4", {"class":"propPrice"}).text.strip()
    print(price)
    address0=item.find_all("span", {"class":"propAddressCollapse"})[0].text.strip()
    print(address0)
    address1=item.find_all("span", {"class":"propAddressCollapse"})[1].text.strip()
    print(address1)
    try:
         bed= item.find("span", {"class":"infoBed"}).text
         print(bed)
    except: 
        print (None) 
    for column_group in item.find_all("div", {"class":"columnGroup"}):
        #print(column_group)
        for feature_group, feature_name in zip(column_group.find_all("span", {"class":"featureGroup"}), column_group.find_all("span", {"class": "featureName"})):
            #print(feature_group.text, feature_name.text)
            if "Lot Size" in feature_group.text:
                print(feature_name.text)
           


3
$725,000
0 Gateway
Rock Springs, WY 82901
None
$452,900
1003 Winchester Blvd.
Rock Springs, WY 82901
4 Beds
0.21 Acres
$396,900
600 Talladega
Rock Springs, WY 82901
5 Beds
$389,900
3239 Spearhead Way
Rock Springs, WY 82901
4 Beds
Under 1/2 Acre, 
$254,000
522 Emerald Street
Rock Springs, WY 82901
3 Beds
Under 1/2 Acre, 
$252,900
1302 Veteran's Drive
Rock Springs, WY 82901
4 Beds
0.27 Acres
$210,000
1021 Cypress Cir
Rock Springs, WY 82901
4 Beds
Under 1/2 Acre, 
$209,000
913 Madison Dr
Rock Springs, WY 82901
3 Beds
Under 1/2 Acre, 
$199,900
1344 Teton Street
Rock Springs, WY 82901
3 Beds
Under 1/2 Acre, 
$196,900
4 Minnies Lane
Rock Springs, WY 82901
3 Beds
2.02 Acres


Saving Extracted data to panda dataframe

In [52]:
l=[]
for item in all:
    d={}
    try:
        d["Address"]= item.find_all("span", {"class":"propAddressCollapse"})[0].text.strip()
    except:
        d["Address"]=None
    try:
        d["Locality"]= item.find_all("span", {"class":"propAddressCollapse"})[0].text.strip()
    except:
        d["Locality"]=None
    d["Price"]= item.find("h4", {"class":"propPrice"}).text.strip()
    try:
        d["Beds"]= item.find("span", {"class": "infoBed"}).text.strip()
    except:
        d["Beds"]=None
    try:
        d["Area"]= item.find("span", {"class": "infoSqFt"}).find("b").text.strip()
    except:
        d["Area"]=None
    try:
        d["Full Baths"]= item.find("span", {"class": "infoValueFullBath"}).find("b").text.strip()
    except:
        d["Full Baths"]=None
    try:
        d["Half Baths"]= item.find("span", {"class": "infoValueHalfBath"}).find("b").text.strip()
    except:
        d["Half Baths"]=None

    for column_group in item.find_all("div", {"class":"columnGroup"}):
        #print(column_group)
        for feature_group, feature_name in zip(column_group.find_all("span", {"class":"featureGroup"}), column_group.find_all("span", {"class": "featureName"})):
            #print(feature_group.text, feature_name.text)
            if "Lot Size" in feature_group.text:
                d["Lot Size"]=feature_name.text
    
    l.append(d)

In [53]:
import pandas
df=pandas.DataFrame(l)
df

,Address,Locality,Price,Beds,Area,Full Baths,Half Baths,Lot Size
0,0 Gateway,0 Gateway,"$725,000",None,None,None,None,NaN
1,1003 Winchester Blvd.,1003 Winchester Blvd.,"$452,900",4 Beds,None,4,None,0.21 Acres
2,600 Talladega,600 Talladega,"$396,900",5 Beds,"3,154",3,None,NaN
3,3239 Spearhead Way,3239 Spearhead Way,"$389,900",4 Beds,"3,076",3,1,"Under 1/2 Acre,"
4,522 Emerald Street,522 Emerald Street,"$254,000",3 Beds,"1,172",3,None,"Under 1/2 Acre,"
5,1302 Veteran's Drive,1302 Veteran's Drive,"$252,900",4 Beds,"1,932",2,None,0.27 Acres
6,1021 Cypress Cir,1021 Cypress Cir,"$210,000",4 Beds,"1,676",3,None,"Under 1/2 Acre,"
7,913 Madison Dr,913 Madison Dr,"$209,000",3 Beds,"1,344",2,None,"Under 1/2 Acre,"
8,1344 Teton Street,1344 Teton Street,"$199,900",3 Beds,"1,920",2,None,"Under 1/2 Acre,"
9,4 Minnies Lane,4 Minnies Lane,"$196,900",3 Beds,"1,664",2,None,2.02 Acres


In [54]:
# saving to csv file
df.to_csv("output.csv")

Crawing through multiple pages

In [66]:
base_url="https://pythonizing.github.io/data/real-estate/rock-springs-wy/LCWYROCKSPRINGS/#t=0&s="
l_all=[]
for page in range(0, int(last_page)*10 ,10):
    print(base_url+str(page)+".html")
    r=requests.get(base_url+str(page)+".html")
    c=r.content
    soup=BeautifulSoup(c,"html.parser")
    all= soup.find_all("div", {"class":"propertyRow"})
    for item in all:
        d={}
        try:
            d["Address"]= item.find_all("span", {"class":"propAddressCollapse"})[0].text.strip()
        except:
            d["Address"]=None
        try:
            d["Locality"]= item.find_all("span", {"class":"propAddressCollapse"})[0].text.strip()
        except:
            d["Locality"]=None
        d["Price"]= item.find("h4", {"class":"propPrice"}).text.strip()
        try:
            d["Beds"]= item.find("span", {"class": "infoBed"}).text.strip()
        except:
            d["Beds"]=None
        try:
            d["Area"]= item.find("span", {"class": "infoSqFt"}).find("b").text.strip()
        except:
            d["Area"]=None
        try:
            d["Full Baths"]= item.find("span", {"class": "infoValueFullBath"}).find("b").text.strip()
        except:
            d["Full Baths"]=None
        try:
            d["Half Baths"]= item.find("span", {"class": "infoValueHalfBath"}).find("b").text.strip()
        except:
            d["Half Baths"]=None

        for column_group in item.find_all("div", {"class":"columnGroup"}):
            #print(column_group)
            for feature_group, feature_name in zip(column_group.find_all("span", {"class":"featureGroup"}), column_group.find_all("span", {"class": "featureName"})):
                #print(feature_group.text, feature_name.text)
                if "Lot Size" in feature_group.text:
                    d["Lot Size"]=feature_name.text
        
        l_all.append(d)
    
    

df_all=pandas.DataFrame(l_all)
df_all

# export to csv file
df_all.to_csv("output_all.csv")

https://pythonizing.github.io/data/real-estate/rock-springs-wy/LCWYROCKSPRINGS/#t=0&s=0.html
https://pythonizing.github.io/data/real-estate/rock-springs-wy/LCWYROCKSPRINGS/#t=0&s=10.html
https://pythonizing.github.io/data/real-estate/rock-springs-wy/LCWYROCKSPRINGS/#t=0&s=20.html
